<a href="https://colab.research.google.com/github/young0jin/CustomsNotifyAnalyzer/blob/main/pdf2csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2
!pip install pdfplumber

In [2]:
import nltk
from nltk.corpus import stopwords
import re

In [3]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [4]:
import requests
import pdfplumber
import pandas as pd
import re

# 다운로드할 PDF 파일 URL 및 저장할 파일 이름 설정
url = "https://www.cbp.gov/sites/default/files/2024-09/vol_58_no_38_complete.pdf"
file_name = "vol_58_no_38_complete.pdf"

# PDF 파일 다운로드
response = requests.get(url)
with open(file_name, 'wb') as file:
    file.write(response.content)

# PDF 파일 열기 및 텍스트 추출
documents = []  # 페이지별 텍스트 저장할 리스트
with pdfplumber.open(file_name) as pdf:
    for page_number, page in enumerate(pdf.pages, start=1):
        text = page.extract_text()
        if text:
            # 줄바꿈을 공백으로 변환하고, 여러 줄바꿈은 문단 구분으로 처리
            cleaned_text = re.sub(r'\n+', ' ', text)  # 여러 줄바꿈을 공백으로 변환
            cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # 다중 공백 정리
            cleaned_text = cleaned_text.strip()  # 양쪽 공백 제거

            # 페이지와 함께 텍스트 저장
            documents.append({"page": page_number, "text": cleaned_text})

# DataFrame으로 변환
df = pd.DataFrame(documents)
df
# CSV 파일로 저장
# csv_file_name = "extracted_text.csv"
# df.to_csv(csv_file_name, index=False)

# print(f"CSV 파일 '{csv_file_name}'로 저장되었습니다.")


,page,text
0,1,U.S. Customs and Border Protection ◆ DEPARTMEN...
1,2,"2 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...."
2,3,"3 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...."
3,4,"4 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...."
4,5,"5 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...."
...,...,...
78,80,"82 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO..."
79,81,"83 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO..."
80,82,"84 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO..."
81,83,"Index Customs Bulletin and Decisions Vol. 58, ..."


In [5]:
pure_ex_text = df["text"].iloc[2]
pure_ex_text

'3 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO. 38, SEPTEMBER 25, 2024 restrictions under 19 U.S.C. 2602(a)(2). On November 18, 2022, the Assistant Secretary for Educational and Cultural Affairs, United States Department of State, after consultation with and recommen- dation by the Cultural Property Advisory Committee, made the de- terminations required under the statute with respect to certain ar- chaeological and ethnological material originating in Yemen that is described in the Designated List set forth below in this document. These determinations include the following: (1) that the cultural patrimony of Yemen is in jeopardy from the pillage of archaeological material, dating from approximately 200,000 B.C. to A.D. 1773, and ethnological material representing Yemen’s cultural heritage, rang- ing in date from approximately A.D. 1517 through 1918 (19 U.S.C. 2602(a)(1)(A)); (2) that the Republic of Yemen Government has taken measures consistent with the Convention to protect its cultu

In [10]:
# ar- chaeological > archaeological
import re
def remove_hypen_space(text):
  cleaned_text = re.sub(r'-\s+', "",text)
  return cleaned_text

In [11]:
df['text'] = df['text'].apply(remove_hypen_space)

추출적 요약 Bart 사용

In [6]:
from transformers import BartForConditionalGeneration, BartTokenizer

In [7]:
model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


- max_length : 생성된 텍스트의 최대 길이
- min_length : 생성된 텍스트의 최소 길이
- length_penalty : 생성된 텍스트의 길이를 조정 (>1.0 : 긴문장, <1.0 : 짧은 문장)
- num_beams : beam_search에서 사용되는 빔의 수 ,보통 3~5, 빔의 수를 늘리면 더 많은 후보 고려 but 계산 비용 증가
- tempearture : 확률 분포의 다양성 증가
- top_k, top_p : 다음 단어 선택에서 무작위성 또는 확률에 따른 제어
- no_repeat_ngram_size : n그램 반복을 방지하여 자연스러운 텍스트 생성

In [12]:
def summarize_text(text):
  inputs = tokenizer([text], max_length = 1024, return_tensors = 'pt')
  summary_ids = model.generate(inputs['input_ids'], max_length = 100, min_length = 40, length_penalty = 0.5, num_beams = 4, early_stopping = True)
  summary = tokenizer.decode(summary_ids[0], skip_special_tokens = True)

  return summary


In [ ]:
df['summary'] = df['text'].apply(summarize_text)

In [14]:
df

,page,text,summary
0,1,U.S. Customs and Border Protection ◆ DEPARTMEN...,The restrictions were originally imposed on an...
1,2,"2 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO....",The CPIA authorizes the imposition of restrict...
2,3,"3 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO....",The United States and the Republic of Yemen si...
3,4,"4 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO....",The Agreement between the U.S. and the Republi...
4,5,"5 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO....",Restricted archaeological material from Yemen ...
...,...,...,...
78,80,"82 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...","On March 19, 2024, New Zealand moved to dissol..."
79,81,"83 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...","The case’s dismissal is issued below, and this..."
80,82,"84 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...",This matter came before the court on a Stipula...
81,83,"Index Customs Bulletin and Decisions Vol. 58, ...","Index Customs Bulletin and Decisions Vol. 58, ..."


In [15]:
df['summary']

,summary
0,The restrictions were originally imposed on an...
1,The CPIA authorizes the imposition of restrict...
2,The United States and the Republic of Yemen si...
3,The Agreement between the U.S. and the Republi...
4,Restricted archaeological material from Yemen ...
...,...
78,"On March 19, 2024, New Zealand moved to dissol..."
79,"The case’s dismissal is issued below, and this..."
80,This matter came before the court on a Stipula...
81,"Index Customs Bulletin and Decisions Vol. 58, ..."


In [20]:
df['summary']

,summary
0,The restrictions were originally imposed on an...
1,The CPIA authorizes the imposition of restrict...
2,The United States and the Republic of Yemen si...
3,The Agreement between the U.S. and the Republi...
4,Restricted archaeological material from Yemen ...
...,...
78,"On March 19, 2024, New Zealand moved to dissol..."
79,"The case’s dismissal is issued below, and this..."
80,This matter came before the court on a Stipula...
81,"Index Customs Bulletin and Decisions Vol. 58, ..."


In [21]:
df.to_csv('summarized_data.csv', index = False)

In [6]:
import pandas as pd
file_path = '/content/summarized_data.csv'
df = pd.read_csv(file_path)

keybert로 키워드 추출

In [4]:
!pip install keybert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 6.3 MB/s eta 0:00:00


In [5]:
from keybert import KeyBERT

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [7]:
sample = df['summary'].iloc[3]

- keyphrase_ngram_range : n-gram의 범위를 지정
- stop_words = 'english' : 불용어 제거
- top_n : 상위 5개의 키워드 추출

In [22]:
kw_model = KeyBERT()
def extract_keywords(text):
  keywords = kw_model.extract_keywords(text, keyphrase_ngram_range = (1,2), stop_words = 'english',use_maxsum = True, top_n = 5)
  sorted_keywords = sorted(keywords, key = lambda x: x[1], reverse = True)
  keywords_only = [kw[0] for kw in sorted_keywords ]
  return keywords_only

In [24]:
df['keywords'] = df['summary'].apply(extract_keywords)

In [25]:
df

,page,text,summary,keywords
0,1,U.S. Customs and Border Protection ◆ DEPARTMEN...,The restrictions were originally imposed on an...,"[imposed emergency, various amendments, 15 202..."
1,2,"2 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO....",The CPIA authorizes the imposition of restrict...,"[material yemen, import restrictions, continue..."
2,3,"3 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO....",The United States and the Republic of Yemen si...,"[republic yemen, bilateral agreement, april 15..."
3,4,"4 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO....",The Agreement between the U.S. and the Republi...,"[republic yemen, categories objects, archaeolo..."
4,5,"5 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO....",Restricted archaeological material from Yemen ...,"[yemen includes, paleolithic neolithic, sasani..."
...,...,...,...,...
78,80,"82 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...","On March 19, 2024, New Zealand moved to dissol...","[new zealand, moved dissolve, 2025, extensions..."
79,81,"83 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...","The case’s dismissal is issued below, and this...","[dolphin easily, remain population, maui, perc..."
80,82,"84 CUSTOMS BULLETIN AND DECISIONS, VOL. 58, NO...",This matter came before the court on a Stipula...,"[dismissal uscit, order voluntary, shepherd co..."
81,83,"Index Customs Bulletin and Decisions Vol. 58, ...","Index Customs Bulletin and Decisions Vol. 58, ...","[import restrictions, ukraine, index customs, ..."


In [26]:
df.to_csv('extract_data.csv', index = False)